Importing Libraries

In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

from sklearn.model_selection import TimeSeriesSplit
from tensorflow.keras.models import Sequential, Model
from tensorflow.keras.layers import LSTM, Input, TimeDistributed, RepeatVector, Dense
from tensorflow.keras.layers import Bidirectional, \
    multiply, concatenate, Flatten, Activation, dot

from sklearn.preprocessing import StandardScaler, MinMaxScaler
from sklearn.model_selection import GridSearchCV
from keras.wrappers.scikit_learn import KerasRegressor
import keras

from sklearn.metrics import mean_absolute_error, mean_squared_error, r2_score
mae, rmse, r2 = mean_absolute_error, mean_squared_error, r2_score

# !pip install tensorflow_addons==0.16.1
import tensorflow_addons as tfa
from tensorflow_addons.optimizers import AdamW

c:\Users\bobby\anaconda3\envs\research\lib\site-packages\scipy\__init__.py:146: UserWarning: A NumPy version >=1.16.5 and <1.23.0 is required for this version of SciPy (detected version 1.23.1
  warnings.warn(f"A NumPy version >={np_minversion} and <{np_maxversion}"
c:\Users\bobby\anaconda3\envs\research\lib\site-packages\tensorflow_addons\utils\ensure_tf_install.py:53: UserWarning: Tensorflow Addons supports using Python ops for all Tensorflow versions above or equal to 2.6.0 and strictly below 2.9.0 (nightly versions are not supported). 
 The versions of TensorFlow you are currently using is 2.9.1 and is not supported. 
Some things might work, some things might not.
If you were to encounter a bug, do not file an issue.
If you want to make sure you're using a tested and supported configuration, either change the TensorFlow version or the TensorFlow Addons's version. 
You can find the compatibility matrix in TensorFlow Addon's readme:
https://github.com/tensorflow/addons
  warnings.war

In [2]:
def readdata(inputcsv, outputcsv):

    idx = pd.IndexSlice
    input_data2 = pd.read_csv(inputcsv, index_col=[0], header=[0,1])
    input_data2.index = pd.to_datetime(input_data2.index)
    input_data2.columns = input_data2.columns.set_levels(input_data2.columns.levels[0].astype('int64'), level=0)
    input_data2.columns = input_data2.columns.set_levels(input_data2.columns.levels[1].astype('string'), level=1)

    ground_truth2 = pd.read_csv(outputcsv, index_col=[0], header=[0,1])
    ground_truth2.index = pd.to_datetime(ground_truth2.index)
    ground_truth2.columns = ground_truth2.columns.set_levels(ground_truth2.columns.levels[0].astype('int64'), level=0)
    ground_truth2.columns = ground_truth2.columns.set_levels(ground_truth2.columns.levels[1].astype('string'), level=1)

    log_transform = lambda x: np.log10(x+1) if x.name[1] == 'tp' else x
    input_data2 = input_data2.apply(log_transform)
    ground_truth2 = ground_truth2.apply(log_transform)

    scaledx = MinMaxScaler()
    scaled_input = scaledx.fit_transform(input_data2.values)
    scaled_input_df = pd.DataFrame(scaled_input, index=input_data2.index, columns=input_data2.columns)

    scaledy = MinMaxScaler()
    scaled_ground = scaledy.fit_transform(ground_truth2.values)
    scaled_ground_df = pd.DataFrame(scaled_ground, index=ground_truth2.index, columns=ground_truth2.columns)

    frames = [scaled_input_df, scaled_ground_df]
    dataset = pd.concat(frames, axis=1)

    train_dataset = dataset.loc['2000-01-01':'2016-12-31']
    test_dataset = dataset.loc['2017-01-01':'2019-12-31']

    return train_dataset, test_dataset, scaledx, scaledy

post_PATH = 'C:\\Users\\bobby\\Documents\\GitHub\\attentionMedium\\postprocessed_data\\'
train_dataset, test_dataset, scaledx, scaledy = readdata(post_PATH+'input_data.csv', post_PATH+'ground_truth.csv')

In [7]:
train_dataset

leadtime          1                                                 2   \
vars             t2m       tp        H         C         E         t2m   
2000-01-01  0.454364  0.414554  0.665413  0.788165  0.382167  0.569784   
2000-01-02  0.498040  0.394784  0.614060  0.910384  0.349166  0.475599   
2000-01-03  0.533352  0.219822  0.588761  0.827255  0.323814  0.562780   
2000-01-04  0.528692  0.439931  0.585007  0.878862  0.310964  0.323332   
2000-01-05  0.365374  0.481476  0.559571  0.899403  0.232568  0.126932   
...              ...       ...       ...       ...       ...       ...   
2016-12-27  0.478379  0.437978  0.575080  0.910384  0.172611  0.224667   
2016-12-28  0.442718  0.296664  0.454412  0.910384  0.081907  0.522516   
2016-12-29  0.479029  0.061326  0.500099  0.905832  0.138873  0.603320   
2016-12-30  0.545704  0.372505  0.600421  0.645576  0.225309  0.587668   
2016-12-31  0.552916  0.522450  0.673749  0.865775  0.298267  0.578814   

leadtime                                            ...        9             \
vars             tp        H         C         E    ...       tp        H     
2000-01-01  0.516938  0.683578  0.992458  0.366075  ...  0.078876  0.478847   
2000-01-02  0.553164  0.636443  0.991829  0.262131  ...  0.412755  0.569225   
2000-01-03  0.477627  0.639143  0.834067  0.269942  ...  0.621661  0.528254   
2000-01-04  0.490424  0.569462  0.981772  0.125560  ...  0.530661  0.525183   
2000-01-05  0.792039  0.480211  0.998114  0.038795  ...  0.361703  0.506658   
...              ...       ...       ...       ...  ...       ...       ...   
2016-12-27  0.449791  0.426003  0.964802  0.018779  ...  0.374733  0.681752   
2016-12-28  0.000000  0.479768  0.888121  0.042124  ...  0.449595  0.640450   
2016-12-29  0.314504  0.634513  0.913262  0.214075  ...  0.227360  0.565322   
2016-12-30  0.460916  0.680944  0.871779  0.271051  ...  0.362376  0.641258   
2016-12-31  0.416016  0.721424  0.973602  0.297106  ...  0.068481  0.537430   

leadtime                              10                                \
vars             C         E         t2m       tp        H         C     
2000-01-01  0.362500  0.267941  0.530388  0.412039  0.538813  0.492143   
2000-01-02  0.657500  0.293947  0.444551  0.521796  0.525433  0.962916   
2000-01-03  0.868750  0.279250  0.368007  0.435447  0.512694  0.969830   
2000-01-04  0.865000  0.227981  0.476429  0.433728  0.545146  0.899434   
2000-01-05  0.650625  0.274030  0.497078  0.088718  0.371129  0.543055   
...              ...       ...       ...       ...       ...       ...   
2016-12-27  0.754375  0.390794  0.608498  0.420226  0.629725  0.920805   
2016-12-28  0.924375  0.234016  0.582628  0.413910  0.544096  0.973602   
2016-12-29  0.998750  0.146430  0.661096  0.340489  0.625849  0.981772   
2016-12-30  0.996875  0.274139  0.558021  0.474441  0.621887  0.989315   
2016-12-31  0.820000  0.248043  0.590371  0.237360  0.629197  0.979887   

leadtime                    0   
vars             E         tp   
2000-01-01  0.360572  0.440304  
2000-01-02  0.246420  0.431659  
2000-01-03  0.254245  0.282224  
2000-01-04  0.274674  0.474193  
2000-01-05  0.120377  0.442982  
...              ...       ...  
2016-12-27  0.348392  0.498704  
2016-12-28  0.162912  0.420299  
2016-12-29  0.278181  0.038538  
2016-12-30  0.222734  0.352531  
2016-12-31  0.363212  0.484940  

[6210 rows x 51 columns]

In [8]:
test_dataset

leadtime          1                                                 2   \
vars             t2m       tp        H         C         E         t2m   
2017-01-01  0.606345  0.221450  0.681078  0.880910  0.327974  0.586197   
2017-01-02  0.529346  0.547792  0.664913  0.908108  0.271549  0.372642   
2017-01-03  0.349326  0.071269  0.550437  0.908677  0.099480  0.745250   
2017-01-04  0.468801  0.136579  0.625138  0.907539  0.217833  0.564897   
2017-01-05  0.487656  0.182539  0.465449  0.811323  0.103705  0.786723   
...              ...       ...       ...       ...       ...       ...   
2019-12-27  0.553681  0.000000  0.462616  0.510839  0.100065  0.595747   
2019-12-28  0.605186  0.000000  0.464586  0.578777  0.084399  0.608687   
2019-12-29  0.595746  0.028346  0.550411  0.292347  0.150488  0.574579   
2019-12-30  0.573165  0.028346  0.545643  0.401991  0.163337  0.585072   
2019-12-31  0.596363  0.182539  0.532268  0.871465  0.176403  0.653204   

leadtime                                            ...        9             \
vars             tp        H         C         E    ...       tp        H     
2017-01-01  0.621545  0.733608  1.000000  0.306133  ...  0.223160  0.588722   
2017-01-02  0.373394  0.604143  0.998743  0.080364  ...  0.124826  0.566352   
2017-01-03  0.271893  0.626964  0.910748  0.253894  ...  0.499382  0.647539   
2017-01-04  0.017491  0.552883  0.698931  0.162776  ...  0.367352  0.529656   
2017-01-05  0.138327  0.583955  0.470145  0.261534  ...  0.206896  0.359012   
...              ...       ...       ...       ...  ...       ...       ...   
2019-12-27  0.250929  0.545317  0.438718  0.090799  ...  0.013518  0.519140   
2019-12-28  0.004531  0.553155  0.488372  0.112949  ...  0.419513  0.551952   
2019-12-29  0.025648  0.572808  0.528598  0.117963  ...  0.408076  0.606053   
2019-12-30  0.229898  0.552204  0.949717  0.124941  ...  0.321171  0.481893   
2019-12-31  0.243778  0.617990  0.746700  0.224702  ...  0.348421  0.593456   

leadtime                              10                                \
vars             C         E         t2m       tp        H         C     
2017-01-01  0.794375  0.309188  0.653845  0.080839  0.570539  0.996229   
2017-01-02  0.703125  0.241899  0.599699  0.281819  0.552243  0.623507   
2017-01-03  0.707500  0.345578  0.574179  0.410663  0.514682  0.878693   
2017-01-04  0.993125  0.225806  0.589480  0.167621  0.430658  0.997486   
2017-01-05  0.999375  0.083599  0.569789  0.343082  0.428890  0.826524   
...              ...       ...       ...       ...       ...       ...   
2019-12-27  0.873750  0.192135  0.591360  0.161005  0.514390  0.959774   
2019-12-28  0.862500  0.176839  0.578347  0.409672  0.530884  0.954745   
2019-12-29  0.888125  0.244527  0.573716  0.369496  0.587936  0.949717   
2019-12-30  0.890000  0.129050  0.547839  0.145117  0.533479  0.989943   
2019-12-31  0.836250  0.244672  0.645868  0.150578  0.469133  0.883721   

leadtime                    0   
vars             E         tp   
2017-01-01  0.304604  0.230563  
2017-01-02  0.248502  0.511228  
2017-01-03  0.212153  0.361810  
2017-01-04  0.151579  0.310881  
2017-01-05  0.136759  0.143848  
...              ...       ...  
2019-12-27  0.192168  0.035049  
2019-12-28  0.166381  0.020342  
2019-12-29  0.259198  0.075689  
2019-12-30  0.172413  0.101514  
2019-12-31  0.128259  0.282224  

[1095 rows x 51 columns]

In [4]:
date_index = pd.date_range('2000-01-01','2016-12-22',freq='D') # changed to 2000-01-01 from 2000-01-10
break_index = [0]+[list(date_index).index(pd.to_datetime('%s-12-31'%year))+1
 for year in range(2000,2016)] + [len(date_index)]

def split_by_year(freq_year, break_index):
    # freq_year = 2
    end_index = 0
    tscv = []
    while True:
        start_index = end_index
        if start_index + freq_year >= len(break_index):
            break
        end_index = min(start_index+2*freq_year, len(break_index)-1)
        tscv.append((list(range(break_index[start_index],break_index[start_index+freq_year])),
                     list(range(break_index[start_index+freq_year],break_index[end_index]))))
    return tscv

freq_year = 2 # freq_year < total year/2  # 2000-2001 (2 years) test 2 years x4
tscv = split_by_year(freq_year, break_index)

def get_xy(series, time_step, n_feature):
    x = series.iloc[:,:-1].T.unstack(level=0).T.values.reshape(len(series),time_step,n_feature) # time_step will be 10
    y = pd.concat([series.iloc[:,-1].shift(-i) for i in range(time_step)], axis=1).dropna(axis=0, how='any').values
    y = y.reshape(y.shape[0],y.shape[1],1)
    x = x[:y.shape[0],:,:]
    return x, y

time_step, n_features = 10, 5
train_x, train_y = get_xy(train_dataset, time_step, n_features)
test_x, test_y = get_xy(test_dataset, time_step, n_features)

input_train = Input(shape=(train_x.shape[1], train_x.shape[2]))
output_train = Input(shape=(train_y.shape[1], train_y.shape[2]))

# gridsearchCV does not take in 3D shape as inputs, only 2D
train_x = train_x.reshape(train_x.shape[0], train_x.shape[1]*train_x.shape[2])
train_y = train_y.reshape(train_y.shape[0], train_y.shape[1]*train_y.shape[2])

In [12]:
pd.DataFrame(tscv)

,0,1
0,"[0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13,...","[731, 732, 733, 734, 735, 736, 737, 738, 739, ..."
1,"[1461, 1462, 1463, 1464, 1465, 1466, 1467, 146...","[2192, 2193, 2194, 2195, 2196, 2197, 2198, 219..."
2,"[2922, 2923, 2924, 2925, 2926, 2927, 2928, 292...","[3653, 3654, 3655, 3656, 3657, 3658, 3659, 366..."
3,"[4383, 4384, 4385, 4386, 4387, 4388, 4389, 439...","[5114, 5115, 5116, 5117, 5118, 5119, 5120, 512..."


In [15]:
print(train_x.shape)
print(train_y.shape)

(6201, 50)
(6201, 10)


In [ ]:
from tensorflow.python.ops.gen_math_ops import square
from tensorflow.keras.optimizers import Adam
from sklearn.metrics import mean_squared_error as mse
from sklearn.metrics import make_scorer

# Loss function with weights based on amplitude of y_true
import tensorflow as tf
from keras import backend as K

np.random.seed(42)
tf.random.set_seed(42)

def my_MSE_weighted2(y_true,y_pred):
    return K.mean(tf.multiply(tf.exp(tf.multiply(2.0, y_true)), tf.square(tf.subtract(y_pred, y_true))))

# scoring
def my_custom_eval_func(y_true, y_pred):
    # Remove 3D array warning
    if len(y_pred.shape) == 3:
        y_pred = y_pred.reshape(y_pred.shape[:-1])
    return rmse(y_true, y_pred, squared=False)

myenvEstimator  = make_scorer(my_custom_eval_func, greater_is_better=False)
#-------------------------------------------------------------------------------

from sklearn.base import BaseEstimator, ClassifierMixin
class mylstm(BaseEstimator, ClassifierMixin):
    def __init__(self, n_steps=10, n_features=5,
                 activation='relu', optimizer='adam',loss=my_MSE_weighted2,
                 lstm=48, dense=1, verbose=1,
                 epochs=20, batch_size=8):
                 #learning_rate=1e-3, #weight_decay=1e-5):

        # static parameters
        self.n_steps = n_steps
        self.n_features = n_features
        self.verbose = verbose

        # Parameters that can be optimized
        self.activation = activation
        self.optimizer = optimizer
        self.loss = loss
        self.lstm = lstm
        self.dense = dense
        self.epochs = epochs
        self.batch_size = batch_size

        #---------- luong attention ----------------
        encoder_stack_h, encoder_forward_h, encoder_forward_c, encoder_backward_h, encoder_backward_c = Bidirectional(LSTM(self.lstm, activation=self.activation,
                                                                                                                           return_state=True, return_sequences=True))(input_train)
        encoder_last_h = concatenate([encoder_forward_h, encoder_backward_h])
        encoder_last_c = concatenate([encoder_forward_c, encoder_backward_c])

        decoder_input = RepeatVector(output_train.shape[1])(encoder_last_h)
        decoder_stack_h = LSTM(self.lstm*2, activation=self.activation, return_state=False, return_sequences=True)(decoder_input, initial_state=[encoder_last_h, encoder_last_c])
        attention = dot([decoder_stack_h, encoder_stack_h], axes=[2, 2])
        attention = Activation('softmax')(attention)
        context = dot([attention, encoder_stack_h], axes=[2,1])
        decoder_combined_context = concatenate([context, decoder_stack_h])
        out = TimeDistributed(Dense(output_train.shape[2]))(decoder_combined_context)
        self.model = Model(inputs=input_train, outputs=out)
        self.model.compile(optimizer=self.optimizer, loss=self.loss)

    def fit(self, X, y, **kw):
        X = X.reshape(X.shape[0], self.n_steps, self.n_features)

        # Control display output, If parameter `verbose` is given, it will be used.
        # If no `verbose` is given, the default value of the class is used.
        if 'verbose' in kw.keys():
            return self.model.fit(X, y, epochs=self.epochs, batch_size=self.batch_size, **kw)
        else:
            return self.model.fit(X, y, epochs=self.epochs, batch_size=self.batch_size, verbose=self.verbose, **kw)


    def predict(self, X, **kw):
        X = X.reshape(X.shape[0], self.n_steps, self.n_features)

        if 'verbose' in kw.keys():
            return self.model.predict(X, **kw)
        else:
            return self.model.predict(X, verbose=self.verbose, **kw)

    def score(self, X, y, **kw):
        X = X.reshape(X.shape[0], self.n_steps, self.n_features)

        # Control display output
        if 'verbose' in kw.keys():
            return self.model.evaluate(X, y, **kw)
        else:
            return self.model.evaluate(X, y, verbose=self.verbose, **kw)

In [ ]:
opt = Adam(learning_rate=1e-4)

parameters = {'activation':('relu','tanh'), 'lstm':[32,48,64,80,100,128,144,160], 'loss':(my_MSE_weighted2, 'mse')} # Actual calculation

tscv = split_by_year(freq_year, break_index)
clf = GridSearchCV(mylstm(verbose=1, epochs=20, batch_size=8, optimizer=opt), parameters, cv=tscv, scoring=myenvEstimator)

clf.fit(train_x, train_y)

In [ ]:
results_df = pd.DataFrame(clf.cv_results_)
results_df['mean_test_score'] = results_df['mean_test_score'].abs()
results_df = results_df.sort_values(by=["rank_test_score"])
results_df = results_df.set_index(
    results_df["params"].apply(lambda x: "_".join(str(val) for val in x.values()))
).rename_axis("kernel")
results_df[["params", "rank_test_score", "mean_test_score", "std_test_score"]]

model_df = results_df[["params", "rank_test_score", "mean_test_score", "std_test_score"]]
# model_df.to_csv('tunings_mse2_biadam.csv')

In [ ]:
clf.cv_results_

{'mean_fit_time': array([48.25649691, 35.09181929, 35.86329842, 35.25014049, 36.32160974,
        39.80626363, 40.09244198, 38.76100546, 39.58778954, 41.10896057,
        42.42620128, 43.13410336, 44.47147024, 44.06862843, 44.73675287,
        43.60585213, 56.02843034, 45.176467  , 56.78962451, 56.74436426,
        47.26439083, 57.05959105, 48.17155099, 48.85460126, 68.1179077 ,
        59.80541694, 59.39382249, 77.48450124, 59.83895636, 68.34578526,
        59.75040925, 59.13266689]),
 'mean_score_time': array([0.75094962, 0.68144286, 0.72104585, 0.8548755 , 0.70880604,
        0.78209698, 0.76868701, 0.73193061, 0.71389496, 0.75992006,
        0.73628891, 0.76536262, 1.02915251, 0.91230369, 0.82881784,
        0.77171475, 0.79725552, 0.93370324, 0.82907778, 0.78102207,
        0.84719265, 0.79590333, 0.85109013, 0.80885816, 0.82633185,
        0.74711382, 0.78590304, 0.94416595, 0.82948768, 0.88558722,
        0.95583731, 0.76971895]),
 'mean_test_score': array([-0.17140935, -0.16946